In [1]:
%matplotlib inline

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys

lib_path = os.path.join('../src/lib')
if not lib_path in sys.path:
    sys.path.append(lib_path)

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data
from fhd_opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

In [3]:
args = ['eldet']
opt = opts().parse(args)
opt.data_dir = '/data/cc/Data/CHD/detection/'
opt.num_classes = 2
opt.dataset = 'coco_fhd'

torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /data/cc/workspace/Repository/CenterNet/scripts/../../exp/eldet/default
heads {'hm': 2, 'reg': 2, 'l': 1, 'ratio_al': 1, 'ratio_ba': 1, 'theta': 1}
Namespace(K=2, aggr_weight=0.0, agnostic_ex=False, arch='dla_34', aug_ddd=0.5, aug_rot=0, batch_size=32, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[32], data_dir='/data/cc/Data/CHD/detection/', dataset='coco_fhd', debug=0, debug_dir='/data/cc/workspace/Repository/CenterNet/scripts/../../exp/eldet/default/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, ellipse_reg_weight=1, ellipse_weight=1, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/data/cc/workspace/Repository/CenterNet/scripts/../../exp/eldet', exp_id='default', fix_res=True, flip=0.5, flip_test=False, gpus=[

In [4]:
opt.heads

{'hm': 2, 'reg': 2, 'l': 1, 'ratio_al': 1, 'ratio_ba': 1, 'theta': 1}

In [5]:
logger = Logger(opt)

os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

In [6]:
print('Creating model...')
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(
      model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

Creating model...


In [7]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [8]:
print('Setting up data...')
val_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'val'), 
    batch_size=1, 
    shuffle=False,
    num_workers=1,
    pin_memory=True
)

if opt.test:
    _, preds = trainer.val(0, val_loader)
    val_loader.dataset.run_eval(preds, opt.save_dir)

train_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'train'), 
    batch_size=opt.batch_size, 
    shuffle=True,
    num_workers=opt.num_workers,
    pin_memory=True,
    drop_last=True
)


Setting up data...
==> initializing Fetal Heart Disease val data.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loaded val 100 samples
==> initializing Fetal Heart Disease train data.
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loaded train 424 samples


In [9]:
logger = Logger(opt)
print('Starting training...')
best = 1e10
for epoch in range(start_epoch + 1, opt.num_epochs + 1):
    mark = epoch if opt.save_all else 'last'
    log_dict_train, _ = trainer.train(epoch, train_loader)
    logger.write('epoch: {} |'.format(epoch))
    print('epoch: {} |'.format(epoch))
    for k, v in log_dict_train.items():
        logger.scalar_summary('train_{}'.format(k), v, epoch)
        logger.write('{} {:8f} | '.format(k, v))
    if opt.val_intervals > 0 and epoch % opt.val_intervals == 0:
        save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(mark)), 
                 epoch, model, optimizer)
        with torch.no_grad():
            log_dict_val, preds = trainer.val(epoch, val_loader)
        for k, v in log_dict_val.items():
            logger.scalar_summary('val_{}'.format(k), v, epoch)
            logger.write('{} {:8f} | '.format(k, v))
        if log_dict_val[opt.metric] < best:
            best = log_dict_val[opt.metric]
            save_model(os.path.join(opt.save_dir, 'model_best.pth'), 
                   epoch, model)
    else:
        save_model(os.path.join(opt.save_dir, 'model_last.pth'), 
                 epoch, model, optimizer)
    logger.write('\n')
    if epoch in opt.lr_step:
        save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(epoch)), 
                 epoch, model, optimizer)
        lr = opt.lr * (0.1 ** (opt.lr_step.index(epoch) + 1))
        print('Drop LR to', lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
#     break
logger.close()

Starting training...


/home/cc/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/cc/miniconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/cc/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([32, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 1 |
epoch: 2 |
epoch: 3 |


KeyboardInterrupt: 

In [ ]:
for iter_id, batch in enumerate(train_loader): # len(batch) = 6
    break

In [ ]:
batch['wh'].shape, batch['reg'].shape, batch['l'].shape, batch['ratio_al'].shape

In [ ]:
model
output = model(batch['input'].cuda())

In [ ]:
output[0]['reg'].shape

In [ ]:
idx = 0
batch

In [ ]:
F.l1_loss(pred * mask, target * mask, size_average=False)

In [ ]:
# idx += 1
plt.imshow(batch['input'][idx][1, :], cmap='gray')

In [ ]:
batch['hm'][idx][0, :].min()

In [ ]:
idx += 1
img = np.array(batch['input'][idx][0, :])
w, h = batch['wh'][idx][0]
cy, cx = np.where(batch['hm'][idx][0, :]==1.)
cx, cy = 4 * cx[0], 4 * cy[0]
l = batch['l'][idx][0]
# a = batch['a'][idx][0]
# b = batch['b'][idx][0]
a = batch['ratio_al'][idx][0] * l
b = batch['ratio_ba'][idx][0] * a
theta = batch['theta'][idx][0]
angle = int(theta * 180)
lung_pt1 = (int(cx - 0.5 * l), int(cy - 0.5 * l))
lung_pt2 = (int(cx + 0.5 * l), int(cy + 0.5 * l))
img = cv2.rectangle(img=img, pt1=lung_pt1, pt2=lung_pt2, color=(0.5), thickness=5)
img = cv2.ellipse(img, (cx, cy), (int(a), int(b)), angle, 0.0, 360.0, (1), thickness=3)
# l = cv2.rectangle(img=l, pt1=heart_pt1, pt2=heart_pt2, color=(2), thickness=5)

w, h = batch['wh'][idx][1]
cy, cx = np.where(batch['hm'][idx][1, :]==1.)
cx, cy = 4 * cx[0], 4 * cy[0]
l = batch['l'][idx][1]
# a = batch['a'][idx][0]
# b = batch['b'][idx][0]
a = batch['ratio_al'][idx][1] * l
b = batch['ratio_ba'][idx][1] * a
theta = batch['theta'][idx][1]
angle = int(theta * 180)
heart_pt1 = (int(cx - 0.5 * l), int(cy - 0.5 * l))
heart_pt2 = (int(cx + 0.5 * l), int(cy + 0.5 * l))
img = cv2.rectangle(img=img, pt1=heart_pt1, pt2=heart_pt2, color=(1.5), thickness=5)
img = cv2.ellipse(img, (cx, cy), (int(a), int(b)), angle, 0.0, 360.0, (2), thickness=3)

plt.imshow(img)

In [ ]:
mse_loss = torch.nn.MSELoss()

In [ ]:
torch.tensor(5)

In [ ]:
mse_loss(r_a ** 2 * (1 + r_b ** 2), torch.tensor(0.25))

In [ ]:
r_a = batch['ratio_al'][idx][1]
r_b = batch['ratio_ba'][idx][1]
r_a ** 2 * (1 + r_b ** 2)

In [ ]:
img = np.array(batch['input'][idx][0, :])
w, h = batch['wh'][idx][0]
cy, cx = np.where(batch['hm'][idx][0, :]==1.)
cx, cy = 4 * cx[0], 4 * cy[0]
l = batch['l'][idx][0]
a = batch['a'][idx][0]
b = batch['b'][idx][0]
theta = batch['theta'][idx][0]
angle = int(theta * 180)
lung_pt1 = (int(cx - 0.5 * l), int(cy - 0.5 * l))
lung_pt2 = (int(cx + 0.5 * l), int(cy + 0.5 * l))
img = cv2.rectangle(img=img, pt1=lung_pt1, pt2=lung_pt2, color=(1), thickness=5)
img = cv2.ellipse(img, (cx, cy), (int(a), int(b)), angle, 0.0, 360.0, (1), thickness=3)
# l = cv2.rectangle(img=l, pt1=heart_pt1, pt2=heart_pt2, color=(2), thickness=5)
plt.imshow(img)

In [ ]:
batch['theta'].shape

In [ ]:
w, h = np.where(batch['hm'][idx][1, :]==1.)
w, h

In [ ]:
batch['wh'][idx][1]

In [ ]:
6333 // 128, 6333 % 128